# Setup

In [ ]:
import pandas as pd
import numpy as np

import os
import glob
import requests

import seaborn as sns
import matplotlib.pyplot as plt

import geopandas as gp

## Import the preprocessed dataframes

In [ ]:
#Note: If you do not have the preprocessed files in the ../data/processed directory things will not work and some states will be missing.
# You need to run the state file to make this happen!

byCountyDf = pd.concat(map(pd.read_csv, glob.glob('../data/processed/by_county/*')))
byStateDf = pd.concat(map(pd.read_csv, glob.glob('../data/processed/by_state/*')))
excessCdcDeathsDf = pd.read_csv("https://data.cdc.gov/api/views/muzy-jte6/rows.csv?accessType=DOWNLOAD")

## Setup data types for dataframes

In [ ]:
byCountyDf.dtypes

In [ ]:
byCountyDf['Date']   = pd.to_datetime(byCountyDf['Date'])
byCountyDf['State'] = byCountyDf['State'].astype('category')
byCountyDf['County'] = byCountyDf['County'].astype('category')

In [ ]:
byCountyDf.dtypes

In [ ]:
byStateDf.dtypes

In [ ]:
byStateDf['Date']   = pd.to_datetime(byStateDf['Date'])
byStateDf['State'] = byStateDf['State'].astype('category')

In [ ]:
byStateDf.dtypes